# Scripts que devuelve las diferencias entre los ficheros

Importamos

In [1]:
import json
import os
import pandas as pd
from difflib import SequenceMatcher
import jellyfish
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.distance import great_circle
import folium
from folium.plugins import MarkerCluster
from convertbng.util import convert_bng, convert_lonlat, convert_etrs89_to_lonlat
import csv

In [2]:
#Center coordinates
SF_COORDINATES = (40.4168, -3.7038)

# for speed purposes
MAX_RECORDS = 10

In [3]:
def get_id(x):
    try:
        for i in range(len(ids)):
            point = [lat[i],lon[i]]
            if getDistance_meters(x,point) <3:
                ayu[i] = x
                return ids[i]
        return 'No match'
    except:
        return 'No match'

def get_close(x):
    try:
        for i in range(len(ids)):
            point = [lat[i],lon[i]]
            if getDistance_meters(x,point) <50:
                ayu[i] = x
                return ids[i]
        return 'No match'
    except:
        return 'No match'
    
def get_lat(x):
    try:
        ind = ids.index(x)
        return float(lat[ind])
    except:
        return None

def get_lon(x):
    try:
        ind = ids.index(x)
        return float(lon[ind])
    except:
        return None

def get_name(x):
    try:
        ind = ids.index(x)
        return name[ind]
    except:
        return None

def get_score(x):
    return getDistance_meters([x.lat_OSM, x.lon_OSM],[x.LATITUD, x.LONGITUD])



In [4]:
def getDistance_meters(x,y):
    try:
        return great_circle(x, y).kilometers*1000.0
    except:
        return 0

In [5]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/data/')
os.listdir()

['.DS_Store',
 '20170614_BANCOS.csv',
 '20170614_PAPELERAS.csv',
 '20171110_InventarioFuentes.csv',
 'anexo i monumentos.xls',
 'artwork.geojson',
 'bench.geojson',
 'drinking_water.geojson',
 'indice.csv',
 'memorial.geojson',
 'monument.geojson',
 'street_lamp.geojson',
 'waste_basket.geojson']

Creamos un data para los datos de OSM

In [6]:
AYU_fichero = '20170614_PAPELERAS.csv'
OSM_fichero = 'waste_basket.geojson'

In [7]:
with open(OSM_fichero) as f:
    OSM = json.load(f)
f.close()

In [8]:
OSM['features'][0]['properties']

{'@id': 'node/1252641337', 'amenity': 'waste_basket'}

Create searching arrays

In [9]:
ids = [OSM['features'][i]['id'] for i in range(len(OSM['features']))]
lat = [OSM['features'][i]['geometry']['coordinates'][1] for i in range(len(OSM['features']))]
lon = [OSM['features'][i]['geometry']['coordinates'][0] for i in range(len(OSM['features']))]
#name = [OSM['features'][i]['properties']['name'] for i in range(len(OSM['features']))]
ayu = ids[:]

Add ayuntamiento data

In [10]:
AYU = pd.read_csv(AYU_fichero, encoding='latin-1', delimiter=';',index_col=0)
AYU.columns = [x.upper() for x in AYU.columns.values]

if AYU_fichero == '20171110_InventarioFuentes.csv':
    AYU.columns = ['ZONA VERDE / VIA PUBLICA','DISTRITO','DIRECCION',
                   'COMPLEMENTO DE DIRECCI”N','X_ETRS89','Y_ETRS89','SISTEMA DE COORDENADAS',
                   'ESTADO','OBSERVACIONES','FECHA DATOS','LONGITUD','LATITUD']

try:
    AYU = AYU[AYU.LATITUD != 'Error']
    AYU = AYU[AYU.LONGITUD != 'Error']
except:
    print('No errors!')
AYU.LATITUD = AYU.LATITUD.astype('float32')
AYU.LONGITUD = AYU.LONGITUD.astype('float32')
AYU = AYU.sort_values('LATITUD')
AYU['id_OSM'] = pd.Series()
AYU['lat_OSM'] = pd.Series()
AYU['lon_OSM'] = pd.Series()
AYU['coord_OSM'] = pd.Series()
AYU['name_OSM'] = pd.Series()
AYU['score_OSM'] = pd.Series()
AYU['X_ETRS89'] = AYU['X_ETRS89'].apply(lambda x: float(x.replace(',','.')))
AYU['Y_ETRS89'] = AYU['Y_ETRS89'].apply(lambda x: float(x.replace(',','.')))
AYU['coord'] = list(zip(AYU.LATITUD, AYU.LONGITUD))
AYU.tail(2)

/Users/raimundoabrillopez/anaconda/lib/python3.5/site-packages/pandas/core/ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


No errors!


BARRIO                              DIRECCION   MODELO  \
DISTRITO                                                                        
FUENCARRAL-EL PARDO  EL PARDO  CARRETERA EL PARDO A COLMENAR VIEJO,9   50_LTS   
FUENCARRAL-EL PARDO  EL PARDO  CARRETERA EL PARDO A COLMENAR VIEJO,9  120_LTS   

                        X_ETRS89     Y_ETRS89  LONGITUD    LATITUD  id_OSM  \
DISTRITO                                                                     
FUENCARRAL-EL PARDO  433508.4912  4487584.577  -3.78513  40.536339     NaN   
FUENCARRAL-EL PARDO  433472.8068  4487667.043  -3.78556  40.537079     NaN   

                     lat_OSM  lon_OSM  coord_OSM  name_OSM  score_OSM  \
DISTRITO                                                                
FUENCARRAL-EL PARDO      NaN      NaN        NaN       NaN        NaN   
FUENCARRAL-EL PARDO      NaN      NaN        NaN       NaN        NaN   

                                   coord  
DISTRITO                                  
FUENCARRAL-EL PARDO  (40.5363, -3.78513)  
FUENCARRAL-EL PARDO  (40.5371, -3.78556)

In [11]:
AYU.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90533 entries, VILLAVERDE to FUENCARRAL-EL PARDO
Data columns (total 14 columns):
BARRIO       90525 non-null object
DIRECCION    69963 non-null object
MODELO       88199 non-null object
X_ETRS89     90533 non-null float64
Y_ETRS89     90533 non-null float64
LONGITUD     90533 non-null float32
LATITUD      90533 non-null float32
id_OSM       0 non-null float64
lat_OSM      0 non-null float64
lon_OSM      0 non-null float64
coord_OSM    0 non-null float64
name_OSM     0 non-null float64
score_OSM    0 non-null float64
coord        90533 non-null object
dtypes: float32(2), float64(8), object(4)
memory usage: 9.7+ MB


In [12]:
AYU.id_OSM = AYU.coord.apply(lambda x: get_id(x)) 
AYU_NO = AYU[AYU.id_OSM == 'No match']
AYU_OK = AYU[AYU.id_OSM != 'No match']

In [13]:
AYU.tail(2)

BARRIO                              DIRECCION   MODELO  \
DISTRITO                                                                        
FUENCARRAL-EL PARDO  EL PARDO  CARRETERA EL PARDO A COLMENAR VIEJO,9   50_LTS   
FUENCARRAL-EL PARDO  EL PARDO  CARRETERA EL PARDO A COLMENAR VIEJO,9  120_LTS   

                        X_ETRS89     Y_ETRS89  LONGITUD    LATITUD    id_OSM  \
DISTRITO                                                                       
FUENCARRAL-EL PARDO  433508.4912  4487584.577  -3.78513  40.536339  No match   
FUENCARRAL-EL PARDO  433472.8068  4487667.043  -3.78556  40.537079  No match   

                     lat_OSM  lon_OSM  coord_OSM  name_OSM  score_OSM  \
DISTRITO                                                                
FUENCARRAL-EL PARDO      NaN      NaN        NaN       NaN        NaN   
FUENCARRAL-EL PARDO      NaN      NaN        NaN       NaN        NaN   

                                   coord  
DISTRITO                                  
FUENCARRAL-EL PARDO  (40.5363, -3.78513)  
FUENCARRAL-EL PARDO  (40.5371, -3.78556)

In [14]:
AYU_CLOSE = AYU_NO.copy()
AYU_CLOSE.id_OSM = AYU_NO.coord.apply(lambda x: get_close(x))
AYU_NO = AYU_CLOSE[AYU_CLOSE.id_OSM == 'No match']
AYU_CLOSE = AYU_CLOSE[AYU_CLOSE.id_OSM != 'No match']

In [15]:
AYU = pd.concat([AYU_OK, AYU_CLOSE, AYU_NO])

In [16]:
AYU.lat_OSM = AYU.id_OSM.apply(lambda x: get_lat(x))
AYU.lon_OSM = AYU.id_OSM.apply(lambda x: get_lon(x))
AYU['coord'] = list(zip(AYU.lat_OSM, AYU.lon_OSM))
AYU.score_OSM = AYU.apply(lambda x: get_score(x), axis=1)
AYU['coord_OSM'] = list(zip(AYU.lat_OSM, AYU.lon_OSM))

try:
    AYU_CLOSE.lat_OSM = AYU_CLOSE.id_OSM.apply(lambda x: get_lat(x))
    AYU_CLOSE.lon_OSM = AYU_CLOSE.id_OSM.apply(lambda x: get_lon(x))
    AYU_CLOSE['coord'] = list(zip(AYU_CLOSE.lat_OSM, AYU_CLOSE.lon_OSM))
    AYU_CLOSE.score_OSM = AYU_CLOSE.apply(lambda x: get_score(x), axis=1)
    AYU_CLOSE['coord_OSM'] = list(zip(AYU_CLOSE.lat_OSM, AYU_CLOSE.lon_OSM))
except:
    print('No close')

AYU_NO.lat_OSM = AYU_NO.id_OSM.apply(lambda x: get_lat(x))
AYU_NO.lon_OSM = AYU_NO.id_OSM.apply(lambda x: get_lon(x))
AYU_NO['coord'] = list(zip(AYU_NO.lat_OSM, AYU_NO.lon_OSM))
AYU_NO.score_OSM = AYU_NO.apply(lambda x: get_score(x), axis=1)
AYU_NO['coord_OSM'] = list(zip(AYU_NO.lat_OSM, AYU_NO.lon_OSM))

In [17]:
AYU = AYU.sort_values('score_OSM')
#AYU.name_OSM = AYU.id_OSM.apply(lambda x: get_name(x))

In [18]:
AYU_MISSING = []
for i in range(len(ids)):
    if isinstance(ayu[i], str):
        AYU_MISSING.append((ids[i],lat[i],lon[i]))
        
AYU_MISSING = pd.DataFrame(AYU_MISSING)
AYU_MISSING.columns = ['id_OSM','lat_OSM','lon_OSM']

AYU_MISSING = AYU_MISSING.set_index('id_OSM')
AYU_MISSING['coord'] = list(zip(AYU_MISSING.lat_OSM.values, AYU_MISSING.lon_OSM.values))

In [19]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/csv/')

In [20]:
AYU_CLOSE.to_csv(AYU_fichero[:-4]+'-close.csv')
AYU_NO.to_csv(AYU_fichero[:-4]+'-NO-OSM.csv')
AYU_MISSING.to_csv(AYU_fichero[:-4]+'-missing.csv')

## Close to edit

In [21]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/web/')

In [22]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_CLOSE.id_OSM.values)):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(AYU_CLOSE.id_OSM.values[i], parse_html=True),
        location = (AYU_CLOSE.LATITUD.values[i],AYU_CLOSE.LONGITUD.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-edit.html')

## Missing in OSM

In [23]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_NO.id_OSM.values[:100])):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(AYU_NO.index.values[i], parse_html=True),
        location = (AYU_NO.LATITUD.values[i],AYU_NO.LONGITUD.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-missing_OSM.html')

## Missing in Ayu website

In [24]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_MISSING.index.values)):
    folium.Marker(icon=folium.Icon(color='green'),popup=folium.Popup(AYU_MISSING.index.values[i], parse_html=True),
        location = (AYU_MISSING.lat_OSM.values[i],AYU_MISSING.lon_OSM.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-missing_Ayu.html')

In [25]:
mydic = AYU_MISSING.to_dict(orient='index')

In [26]:
mydic.keys()

dict_keys(['node/4124377983', 'node/4619376482', 'node/4619377189', 'node/4123920565', 'node/4891527345', 'node/2803215906', 'node/4124377979', 'node/4123920567', 'node/4123920563', 'node/4124379659', 'node/4619376485', 'node/4891527234', 'node/2821472421', 'node/4123920566', 'node/4124379669', 'node/4619376486', 'node/3285505476', 'node/4123920571', 'node/4891515421', 'node/4619377193', 'node/4123920582', 'node/4124377976', 'node/3428590044', 'node/4632741936', 'node/4124379618', 'node/4891503449', 'node/3786662100', 'node/4124379661', 'node/4124379631', 'node/4619377200', 'node/4123920577', 'node/5035310703', 'node/4619377197', 'node/4124379665', 'node/4891527539', 'node/4456027694', 'node/4619377198', 'node/4619377190', 'node/4619376484', 'node/3353540089', 'node/4635305102', 'node/4891527235', 'node/4632741932', 'node/4123920560', 'node/4123920556', 'node/4124379605', 'node/4619376480', 'node/4619377194', 'node/4123920569', 'node/4123920586', 'node/4891527343', 'node/4123920575', '

In [27]:
import json
print(json.dumps(mydic, indent=4))

{
    "node/4124377983": {
        "LONGITUD": -3.7255671,
        "coord": [
            40.4497355,
            -3.7255671
        ],
        "LATITUD": 40.4497355
    },
    "node/4619376482": {
        "LONGITUD": -3.7078956,
        "coord": [
            40.4656538,
            -3.7078956
        ],
        "LATITUD": 40.4656538
    },
    "node/4619377189": {
        "LONGITUD": -3.7085366,
        "coord": [
            40.4668797,
            -3.7085366
        ],
        "LATITUD": 40.4668797
    },
    "node/4123920565": {
        "LONGITUD": -3.726472,
        "coord": [
            40.4500282,
            -3.726472
        ],
        "LATITUD": 40.4500282
    },
    "node/4891527345": {
        "LONGITUD": -3.7248937,
        "coord": [
            40.4483029,
            -3.7248937
        ],
        "LATITUD": 40.4483029
    },
    "node/2803215906": {
        "LONGITUD": -3.6538094,
        "coord": [
            40.4381268,
            -3.6538094
        ],
        "LA